In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [3]:
train_dataset = datasets.FashionMNIST(root = './fashion', download=True, train=True, transform=transform)
test_dataset = datasets.FashionMNIST(root = './fashion', download=True, train=False, transform=transform)

In [4]:
torch.cuda.is_available()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
train_loader = DataLoader(dataset = train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size=32, shuffle=False)

In [6]:
class Fashion_CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1),  # 28x28 input, 3x3 filter, 8 channel 28x28 feature map
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                                          # 2x2 grid, 8 channel 14x14 feature map
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), 

            nn.Flatten(),
            nn.Linear(16*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

In [7]:
model = Fashion_CNN()
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=0.002)

In [9]:
epochs = 5

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for image, label in train_loader:
        image, label = image.to(device), label.to(device)

        output = model(image)
        loss = criterion(output, label)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        epoch_loss += loss.item()


    print(f'Epoch:{epoch+1}, Loss:{epoch_loss/len(train_loader):.4f}')

Epoch:1, Loss:0.4290
Epoch:2, Loss:0.2878
Epoch:3, Loss:0.2474
Epoch:4, Loss:0.2171
Epoch:5, Loss:0.1967


In [10]:
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for image, label in test_loader:

        output = model(image)
        predict = output.argmax(dim=1)
        correct += (predict == label).sum()
        total += len(label)

    print(f'Accuracy: {correct.item()/total*100:.4f}%')

Accuracy: 90.8500%
